# HIV-CB Event Files.

In [2]:
# Standard imports (i.e., Python builtins) go at the top
from os import listdir
import os.path as op
from glob import glob
import numpy as np

# Now third-party imports
import pandas as pd

# Finally, any local imports would go here

In [3]:
in_folder = '/home/data/nbc/Sutherland_HIVCB/dset'

subject_folders = sorted(glob(op.join(in_folder, 'sub-*')))

# I'm quite sure that there are no files starting with 'sub-', 
# since that would not fit with BIDS, but, just to be safe,
# we can reduce the list to folders only.
subject_folders = [sf for sf in subject_folders if op.isdir(sf)]


# Now let's put these things together

# We need an output directory to save things to
out_dir = '/home/data/nbc/Sutherland_HIVCB/derivatives/afni-processing/preprocessed-data/'

for subject_folder in subject_folders:
    subject_id = op.basename(subject_folder)
    print('Processing {}'.format(subject_id))
    func_folder = op.join(subject_folder, 'func')
    # And grab *all* errorawareness task events files
    events_files = sorted(glob(op.join(func_folder, '*_task-errorawareness_*_events.tsv')))
    
    out_sub_dir = op.join(out_dir, subject_id, 'func')
    
    # Make lists to place all lines in
    go_incorrect_onsets_text = []
    nogo_incorrect_onsets_text = []
    nogo_aware_onsets_text = []
    nogo_unaware_onsets_text = []
    nogo_correct_onsets_text = []
    onebefore_error_onsets_text = []
    twobefore_error_onsets_text = []
    threebefore_error_onsets_text = []
    oneafter_error_onsets_text = []
    twoafter_error_onsets_text = []
    threeafter_error_onsets_text = []
    
    for ev_file in events_files:
        df = pd.read_csv(ev_file, sep='\t')
        
        # Grab incorrect go trials, which are labeled as goIncorrect
        go_incorrect_df = df.loc[df['trial_type_2'].isin(['goIncorrect'])]
        go_incorrect_onsets = go_incorrect_df['onset'].values
        if go_incorrect_onsets.size == 0:
            go_incorrect_onsets = ['*']
        go_incorrect_onsets_text.append('\t'.join([str(num) for num in go_incorrect_onsets]))

        # Grab incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        nogo_incorrect_df = df.loc[df['trial_type_2'].isin(['nogoIncorrectAware', 'nogoIncorrectUnaware'])]
        nogo_incorrect_onsets = nogo_incorrect_df['onset'].values
        if nogo_incorrect_onsets.size == 0:
            nogo_incorrect_onsets = ['*']
        nogo_incorrect_onsets_text.append('\t'.join([str(num) for num in nogo_incorrect_onsets]))
    
        # Grab incorrect nogo aware trials, which are labeled as nogoIncorrectAware
        nogo_aware_df = df.loc[df['trial_type_2'].isin(['nogoIncorrectAware'])]
        nogo_aware_onsets = nogo_aware_df['onset'].values
        if nogo_aware_onsets.size == 0:
            nogo_aware_onsets = ['*']
        nogo_aware_onsets_text.append('\t'.join([str(num) for num in nogo_aware_onsets]))
        
        # Grab incorrect nogo unaware trials, which are labeled as nogoIncorrectUnaware
        nogo_unaware_df = df.loc[df['trial_type_2'].isin(['nogoIncorrectUnaware'])]
        nogo_unaware_onsets = nogo_unaware_df['onset'].values
        if nogo_unaware_onsets.size == 0:
            nogo_unaware_onsets = ['*']
        nogo_unaware_onsets_text.append('\t'.join([str(num) for num in nogo_unaware_onsets]))
    
        # Grab correct nogo trials, which are labeled as nogoCorrect
        nogo_correct_df = df.loc[df['trial_type_2'].isin(['nogoCorrect'])]
        nogo_correct_onsets = nogo_correct_df['onset'].values
        if nogo_correct_onsets.size == 0:
            nogo_correct_onsets = ['*']
        nogo_correct_onsets_text.append('\t'.join([str(num) for num in nogo_correct_onsets]))
        
        idx = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) - 1
        idx2 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) - 1
        cond_idx = np.concatenate((idx, idx2), axis=0)
        cond_idx
        
        # Grab 1 before incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        onebefore_error_df = df.iloc[cond_idx]
        onebefore_error_onsets = onebefore_error_df['onset'].values
        if onebefore_error_onsets.size == 0:
            onebefore_error_onsets = ['*']
        onebefore_error_onsets_text.append('\t'.join([str(num) for num in onebefore_error_onsets]))
    
    
        idx3 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) - 2
        idx4 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) - 2
        cond_idx2 = np.concatenate((idx3, idx4), axis=0)
        cond_idx2
        
        # Grab 2 before incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        twobefore_error_df = df.iloc[cond_idx2]
        twobefore_error_onsets = twobefore_error_df['onset'].values
        if twobefore_error_onsets.size == 0:
            twobefore_error_onsets = ['*']
        twobefore_error_onsets_text.append('\t'.join([str(num) for num in twobefore_error_onsets]))
    
    
        ############## grab 3 before
    
        idx5 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) - 3
        idx6 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) - 3
        cond_idx3 = np.concatenate((idx5, idx6), axis=0)
        cond_idx3
        
        # Grab 3 before incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        threebefore_error_df = df.iloc[cond_idx3]
        threebefore_error_onsets = threebefore_error_df['onset'].values
        if threebefore_error_onsets.size == 0:
            threebefore_error_onsets = ['*']
        threebefore_error_onsets_text.append('\t'.join([str(num) for num in threebefore_error_onsets]))
    
        ############## grab 1 after
    
        idx7 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) + 1
        idx8 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) + 1
        cond_idx4 = np.concatenate((idx7, idx8), axis=0)
        cond_idx4 = np.intersect1d(cond_idx4, np.arange(df.shape[0], dtype=int))
        cond_idx4
        
        # Grab 1 after incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        oneafter_error_df = df.iloc[cond_idx4]
        oneafter_error_onsets = oneafter_error_df['onset'].values
        if oneafter_error_onsets.size == 0:
            oneafter_error_onsets = ['*']
        oneafter_error_onsets_text.append('\t'.join([str(num) for num in oneafter_error_onsets]))
    
         ############## grab 2 after
        
        idx9 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) + 2
        idx10 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) + 2
        cond_idx5 = np.concatenate((idx9, idx10), axis=0)
        cond_idx5 = np.intersect1d(cond_idx5, np.arange(df.shape[0], dtype=int))
        cond_idx5
        
        # Grab 1 after incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        twoafter_error_df = df.iloc[cond_idx5]
        twoafter_error_onsets = twoafter_error_df['onset'].values
        if twoafter_error_onsets.size == 0:
            twoafter_error_onsets = ['*']
        twoafter_error_onsets_text.append('\t'.join([str(num) for num in twoafter_error_onsets]))
    
         ############## grab 3 after
            
        idx11 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) + 3
        idx12 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) + 3
        cond_idx6 = np.concatenate((idx11, idx12), axis=0)
        cond_idx6 = np.intersect1d(cond_idx6, np.arange(df.shape[0], dtype=int))
        cond_idx6
        
        #Grab 1 after incorrect nogo trials, which are labeled as nogoIncorrectAware or nogoIncorrectUnaware
        threeafter_error_df = df.iloc[cond_idx6]  # use iloc bc get_indexer returns row num, not index
        threeafter_error_onsets = threeafter_error_df['onset'].values
        if threeafter_error_onsets.size == 0:
            threeafter_error_onsets = ['*']
        threeafter_error_onsets_text.append('\t'.join([str(num) for num in threeafter_error_onsets]))
    
    #different line for each run
    # Merge list of single-line strings into multiline string
    go_incorrect_onsets_text = '\n'.join(go_incorrect_onsets_text)
    nogo_incorrect_onsets_text = '\n'.join(nogo_incorrect_onsets_text)
    nogo_aware_onsets_text = '\n'.join(nogo_aware_onsets_text)
    nogo_unaware_onsets_text = '\n'.join(nogo_unaware_onsets_text)
    nogo_correct_onsets_text = '\n'.join(nogo_correct_onsets_text)
    onebefore_error_onsets_text = '\n'.join(onebefore_error_onsets_text)
    twobefore_error_onsets_text = '\n'.join(twobefore_error_onsets_text) 
    threebefore_error_onsets_text = '\n'.join(threebefore_error_onsets_text)
    oneafter_error_onsets_text = '\n'.join(oneafter_error_onsets_text)
    twoafter_error_onsets_text = '\n'.join(twoafter_error_onsets_text)
    threeafter_error_onsets_text = '\n'.join(threeafter_error_onsets_text)
                                           
    try:
        #different file for each event type
        go_incorrect_file = op.join(out_sub_dir, 'go_incorrect.1D')
        with open(go_incorrect_file, 'w') as fo:
            fo.write(go_incorrect_onsets_text)

        nogo_incorrect_file = op.join(out_sub_dir, 'nogo_incorrect.1D')
        with open(nogo_incorrect_file, 'w') as fo:
            fo.write(nogo_incorrect_onsets_text)
            
        nogo_aware_file = op.join(out_sub_dir, 'nogo_aware.1D')
        with open(nogo_aware_file, 'w') as fo:
            fo.write(nogo_aware_onsets_text)   
            
        nogo_unaware_file = op.join(out_sub_dir, 'nogo_unaware.1D')
        with open(nogo_unaware_file, 'w') as fo:
            fo.write(nogo_unaware_onsets_text)
            
        nogo_correct_file = op.join(out_sub_dir, 'nogo_correct.1D')
        with open(nogo_correct_file, 'w') as fo:
            fo.write(nogo_correct_onsets_text)  
                                           
        onebefore_error_file = op.join(out_sub_dir, 'onebefore_error.1D')
        with open(onebefore_error_file, 'w') as fo:
            fo.write(onebefore_error_onsets_text)
            
        twobefore_error_file = op.join(out_sub_dir, 'twobefore_error.1D')
        with open(twobefore_error_file, 'w') as fo:
            fo.write(twobefore_error_onsets_text)
            
        threebefore_error_file = op.join(out_sub_dir, 'threebefore_error.1D')
        with open(threebefore_error_file, 'w') as fo:
            fo.write(threebefore_error_onsets_text)
            
        oneafter_error_file = op.join(out_sub_dir, 'oneafter_error.1D')
        with open(oneafter_error_file, 'w') as fo:
            fo.write(oneafter_error_onsets_text)
            
        twoafter_error_file = op.join(out_sub_dir, 'twoafter_error.1D')
        with open(twoafter_error_file, 'w') as fo:
            fo.write(twoafter_error_onsets_text)
            
        threeafter_error_file = op.join(out_sub_dir, 'threeafter_error.1D')
        with open(threeafter_error_file, 'w') as fo:
            fo.write(threeafter_error_onsets_text)   
                                                        
    except:
        print("missing subject")       
            

Processing sub-193
Processing sub-194
Processing sub-198
Processing sub-203
Processing sub-205
missing subject
Processing sub-206
Processing sub-211
Processing sub-212
Processing sub-216
Processing sub-217
Processing sub-218
Processing sub-225
Processing sub-229
Processing sub-237
Processing sub-238
Processing sub-240
Processing sub-241
missing subject
Processing sub-242
Processing sub-244
Processing sub-247
Processing sub-248
Processing sub-250
Processing sub-253
Processing sub-254
Processing sub-257
Processing sub-258
Processing sub-259
Processing sub-263
Processing sub-266
Processing sub-268
Processing sub-273
Processing sub-279
Processing sub-280
Processing sub-282
Processing sub-283
Processing sub-285
Processing sub-287
Processing sub-289
Processing sub-290
Processing sub-291
Processing sub-293
Processing sub-294
Processing sub-295
Processing sub-296
Processing sub-297
Processing sub-299
Processing sub-300
Processing sub-301
Processing sub-302
Processing sub-306
Processing sub-307

In [4]:
idx11 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectAware'].index) + 3
idx12 = df.index.get_indexer_for(df[df['trial_type_2'] == 'nogoIncorrectUnaware'].index) + 3
cond_idx6 = np.concatenate((idx11, idx12), axis=0)
cond_idx6

array([ 10,  17,  32,  36,  41, 108, 114, 122, 195, 203, 211, 101])

In [20]:
df2 = pd.DataFrame(index=[3,5, 6, 7], columns=['hi'], data=np.arange(4))

In [26]:
df2.index.get_indexer_for(df2[df2['hi'] > 2].index)

array([3])

In [27]:
df2

,hi
3,0
5,1
6,2
7,3


In [19]:
df

,onset,duration,response_time,trial_type,trial_type_2,stimulus_word,stimulus_color,trial_accuracy
0,12.007,1.0,0.891,congruent,goCorrect,green,green,1
1,13.507,1.0,0.587,congruent,goCorrect,white,white,1
2,15.007,1.0,0.394,congruent,goCorrect,blue,blue,1
3,16.507,1.0,0.779,congruent,goCorrect,yellow,yellow,1
4,18.007,1.0,0.487,congruent,goCorrect,green,green,1
...,...,...,...,...,...,...,...,...
211,328.516,1.0,0.238,congruent,goCorrect,white,white,1
212,330.016,1.0,1.129,congruent,goCorrect,blue,blue,1
213,331.516,1.0,0.352,congruent,goCorrect,yellow,yellow,1
214,333.016,1.0,0.950,repeat,nogoIncorrectAware,yellow,yellow,0
